## Exercici 1
Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un pandas Dataframe. Explora les dades que conté, i queda’t únicament amb les columnes que consideris rellevants.

In [1]:
import pandas as pd

df = pd.read_csv("DelayedFlights.csv", header=None)
print(df)
print(df.columns)

df.columns = df.iloc[0]
df = df.iloc[1: , :]
print(df.columns)

df_seleccio = df[['Month', 'DayOfWeek', 'ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Origin', 'Dest', 'Distance', 'AirTime','UniqueCarrier']]
print(df_seleccio)

# Mostreig
df_seleccio = df_seleccio.sample(n=50)

/Users/claradv/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5,6,7,8,10,12,13,14,15,16,19,20,21,22,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


                0     1      2           3          4        5           6   \
0              NaN  Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime   
1              0.0  2008      1           3          4   2003.0        1955   
2              1.0  2008      1           3          4    754.0         735   
3              2.0  2008      1           3          4    628.0         620   
4              4.0  2008      1           3          4   1829.0        1755   
...            ...   ...    ...         ...        ...      ...         ...   
1936754  7009710.0  2008     12          13          6   1250.0        1220   
1936755  7009717.0  2008     12          13          6    657.0         600   
1936756  7009718.0  2008     12          13          6   1007.0         847   
1936757  7009726.0  2008     12          13          6   1251.0        1240   
1936758  7009727.0  2008     12          13          6   1110.0        1103   

              7           8              9   ...   

## Exercici 2
Fes un informe complet del data set:

- Resumeix estadísticament les columnes d’interès
- Troba quantes dades faltants hi ha per columna
- Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...)
- Taula de les aerolínies amb més endarreriments acumulats
- Quins són els vols més llargs? I els més endarrerits?
- Etc.

**Dades faltants per columna**

In [ ]:
import numpy as np

for column in df_seleccio:
    print('Columna:', column)
    print('Dades faltants:', df_seleccio[column].isna().sum(), '\n')
    # substiteixo els Nan per zeros per a poder operar amb aquests valors més endavant
    df_seleccio.loc[:, column] = df_seleccio[column].fillna(0)

**Canviar data type per a determinades columnes i així poder fer càlculs**

In [ ]:
df_seleccio.loc[:,'ArrDelay'] = df_seleccio['ArrDelay'].astype(float).astype(int)
df_seleccio.loc[:,'DepDelay'] = df_seleccio['DepDelay'].astype(float).astype(int)
df_seleccio.loc[:,'CarrierDelay'] = df_seleccio['CarrierDelay'].astype(float).astype(int)
df_seleccio.loc[:,'WeatherDelay'] = df_seleccio['WeatherDelay'].astype(float).astype(int)
df_seleccio.loc[:,'NASDelay'] = df_seleccio['NASDelay'].astype(float).astype(int)
df_seleccio.loc[:,'SecurityDelay'] = df_seleccio['SecurityDelay'].astype(float).astype(int)
df_seleccio.loc[:,'LateAircraftDelay'] = df_seleccio['LateAircraftDelay'].astype(float).astype(int)
df_seleccio.loc[:,'Distance'] = df_seleccio['Distance'].astype(float).round(2)
df_seleccio.loc[:,'AirTime'] = df_seleccio['AirTime'].astype(float).astype(int)


**Resum stadístic**

In [ ]:
for column in df_seleccio:
    print(df_seleccio[column].describe(), "\n")

**Crear columnas nuevas**

In [ ]:
df_seleccio.loc[:,'TotalDelay'] = df_seleccio['ArrDelay'] + df_seleccio['DepDelay'] + df_seleccio['CarrierDelay'] + df_seleccio['WeatherDelay'] + df_seleccio['NASDelay'] + df_seleccio['SecurityDelay'] + df_seleccio['LateAircraftDelay']
df_seleccio.loc[:,'AverageSpeed'] = (df_seleccio['Distance'] / df_seleccio['AirTime']).round(2)
df_seleccio.loc[:,'Delayed'] = np.where(df_seleccio['TotalDelay'] > 0, True, False)

**Taula de les aerolínies amb més endarreriments acumulats**

In [ ]:
aerolinies_amb_mes_endarreriments = df_seleccio.groupby(['UniqueCarrier']).size().reset_index(name='NumDelays')
aerolinies_amb_mes_endarreriments = aerolinies_amb_mes_endarreriments.sort_values(by=['NumDelays'], ascending=False).head(3)
print(aerolinies_amb_mes_endarreriments)

**Vols més llargs**

In [ ]:
data = [df_seleccio["Origin"], df_seleccio["Dest"], df_seleccio["AirTime"]]
headers = ["Origin", "Dest", "Duration"]
vols = pd.concat(data, axis=1, keys=headers)
vols_mes_llargs = vols.sort_values(by=['Duration'], ascending=False).head(3)
print(vols_mes_llargs)

**Vols més endarrerits**

In [ ]:
data = [df_seleccio["Origin"], df_seleccio["Dest"], df_seleccio["TotalDelay"]]
headers = ["Origin", "Dest", "Delay"]
vols = pd.concat(data, axis=1, keys=headers)
vols_mes_endarrerits = vols.sort_values(by=['Delay'], ascending=False).head(3)
print(vols_mes_endarrerits)

**Mesos amb més endarreriments**

In [ ]:
mesos_amb_mes_endarreriments = df_seleccio.groupby(['Month']).size().reset_index(name='NumDelays')
mesos_amb_mes_endarreriments.loc[:,'Month'] = pd.to_datetime(mesos_amb_mes_endarreriments['Month'], format='%m').dt.month_name().str.slice(stop=3)
mesos_amb_mes_endarreriments = mesos_amb_mes_endarreriments.sort_values(by=['NumDelays'], ascending=False).head(3)
print(mesos_amb_mes_endarreriments)

**Causes d'endarreriment més freqüents**

In [ ]:
data = df_seleccio['ArrDelay'], df_seleccio['DepDelay'], df_seleccio['CarrierDelay'], df_seleccio['WeatherDelay'], df_seleccio['NASDelay'], df_seleccio['SecurityDelay'], df_seleccio['LateAircraftDelay']
headers = ["ArrDelay", "DepDelay", "CarrierDelay", 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
causas = pd.concat(data, axis=1, keys=headers)
causas_mes_frequents = causas[causas > 0].count().sort_values(ascending=False).head(3)
print(causas_mes_frequents)

## Exercici 3
Exporta el data set net i amb les noves columnes a Excel.

In [ ]:
print(df_seleccio)

df_seleccio.to_excel('DelayedFlights_modificat.xlsx', sheet_name='Sheet1')

![x](variable_categorica.png)
![x](variable_numerica.png)
![x](variables_num_cat.png)
![x](variables_num.png)
![x](tres_variables.png)
![x](quatre_variables_a.png)
![x](quatre_variables_b.png)